# Getting data from Public Health England

In [2]:
import os
import sys
module_path = os.path.abspath(os.pardir)
if module_path not in sys.path:
    sys.path.append(module_path)
    
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import json
from collections import Counter
from pathlib import Path
from urllib import parse
import json
import requests
import pandas as pd

from stream import generate_streams, test_endpoints, get_token, register, generate_streams_from_urls

In [4]:
token = get_token(prod=True)

## Nation

In [7]:
url = 'https://api.coronavirus.data.gov.uk/v2/data?areaType=nation&areaCode=E92000001&metric=cumAdmissions&metric=cumCasesBySpecimenDate&metric=cumOnsDeathsByRegistrationDate&metric=cumVaccinationFirstDoseUptakeByVaccinationDatePercentage&metric=cumVaccinationSecondDoseUptakeByVaccinationDatePercentage&format=csv'
pd.read_csv(url)


,areaCode,areaName,areaType,date,cumAdmissions,cumCasesBySpecimenDate,cumOnsDeathsByRegistrationDate,cumVaccinationFirstDoseUptakeByVaccinationDatePercentage,cumVaccinationSecondDoseUptakeByVaccinationDatePercentage
0,E92000001,England,nation,2022-01-12,NaN,12680151,NaN,79.4,73.3
1,E92000001,England,nation,2022-01-11,566587.0,12644933,NaN,79.3,73.3
2,E92000001,England,nation,2022-01-10,564460.0,12564255,NaN,79.3,73.2
3,E92000001,England,nation,2022-01-09,562280.0,12465313,NaN,79.3,73.2
4,E92000001,England,nation,2022-01-08,560305.0,12392677,NaN,79.3,73.2
...,...,...,...,...,...,...,...,...,...
709,E92000001,England,nation,2020-02-03,NaN,2,NaN,NaN,NaN
710,E92000001,England,nation,2020-02-02,NaN,2,NaN,NaN,NaN
711,E92000001,England,nation,2020-02-01,NaN,2,NaN,NaN,NaN
712,E92000001,England,nation,2020-01-31,NaN,2,NaN,NaN,NaN


## LTLA
### 1. All areas together

In [3]:
with open('../manifest/all-ltla-streams.json') as f:
    all_ltla_streams = json.load(f)

len(all_ltla_streams)

1

In [7]:
test_endpoints(all_ltla_streams, base_url='')

In [8]:
for s in all_ltla_streams:
    register(s, token, prod=True)

### 2. Individual areas

In [9]:
with open('../manifest/ltla-streams.json') as f:
    ltla_base_streams = json.load(f)

print(ltla_base_streams[0])
len(ltla_base_streams)

{'urlCode': 'API_GOVUK', 'endpoint': 'https://api.coronavirus.data.gov.uk/v2/data?areaType=ltla&metric=vaccinationsAgeDemographics&format=json', 'dataType': 'timeseries', 'keywords': ['phe', 'ltla', 'vaccination', 'daily'], 'description': ''}


1

In [11]:
def generate_streams(base_streams, codes):
    streams = []
    for s in base_streams:
        for c in codes:
            scopy = s.copy()
            scopy['endpoint'] = s['endpoint'] + '&areaCode=' + c
            scopy['keywords'] = scopy['keywords'] + [c.lower()]
            streams.append(scopy)
    return streams

In [12]:
ltla_area_codes = ['S12000036']
ltla_streams = generate_streams(ltla_base_streams, ltla_area_codes)
len(ltla_streams)

1

In [13]:
test_endpoints(ltla_streams, base_url='')

In [16]:
for s in ltla_streams:
    register(s, token, prod=True)